In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from collections import OrderedDict

In [2]:
dir_ = '../../output/stage_2_predictions/Keck_Pria_AS_Retest'
complete_df = pd.read_csv('{}/complete_prediction.csv'.format(dir_))

# Show 54 Active Compounds' Corresponding Predicted Values

In [3]:
molecule_ids = complete_df['molecule'].tolist()
actual_labels = complete_df['label'].tolist()
inhibits = complete_df['inhibition'].tolist()
model_names = complete_df.columns[3:]
    
positive_enumerate = filter(lambda x: x[1] == 1, enumerate(actual_labels))
positive_idx = map(lambda x: x[0], positive_enumerate)

In [4]:
columns = ['inhibition']
columns.extend(model_names)
HTML(complete_df[complete_df['label']>0][columns].to_html())

,inhibition,Baseline,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,RandomForest_h,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
367,83.123806,0.63,0.026200,11.627432,10.401617,10.348252,0.052207,6.83,49.549496,10.348237,-0.216938,96.0663,16.759,38.103,8.955,7.17,0.500282,0.499869,0.504866,0.505282,0.511596,0.002003,0.041506,8.292987e-05,5.710541e-05,0.26250,0.263750,0.261125,0.20825,0.209625,0.28625,0.16450,0.160625,0.000011,0.013834,-3.408562,0.307883
1021,72.063678,0.54,0.020902,10.643016,9.375816,9.086127,0.026928,6.72,41.775131,8.748974,0.820223,81.6115,14.474,34.089,8.930,3.36,0.500282,0.522985,0.545548,0.543723,0.554211,0.002003,0.041435,2.490977e-01,2.412799e-01,0.04125,0.039250,0.040313,0.04925,0.054250,0.42625,0.11950,0.118250,0.165573,0.206531,29.097578,85.389641
1528,89.493228,0.32,0.027703,12.084562,10.107620,10.038560,0.039552,6.65,40.496170,11.273830,2.226941,79.9549,16.883,37.892,8.525,4.85,0.710454,0.575938,0.595639,0.592230,0.601834,0.001995,0.041460,2.596361e-04,4.096623e-03,0.62775,0.629375,0.631313,0.55125,0.555625,0.77675,0.61225,0.606500,0.000062,0.025602,4.825109,13.609955
1746,44.850520,0.63,0.136960,13.457431,10.838761,10.761808,0.154201,7.66,43.351162,9.481766,-0.060435,89.0793,27.440,47.976,7.494,8.44,0.500282,0.499869,0.499749,0.500670,0.505504,0.001990,0.041254,8.081133e-09,4.435495e-07,0.23500,0.231750,0.230875,0.23600,0.239375,0.01825,0.23175,0.233625,0.000035,0.012207,-5.790790,-4.602566
1850,93.499335,0.33,0.100197,12.998991,11.054573,10.995590,0.161404,6.55,48.574375,12.967777,1.797404,88.8323,20.722,41.832,9.279,6.81,0.500282,0.511202,0.533921,0.532971,0.542782,0.002001,0.041641,3.407329e-05,1.647828e-04,0.01600,0.017875,0.016125,0.01975,0.021000,0.08550,0.05475,0.052625,0.000329,0.109288,6.454529,12.292480
2547,71.993342,0.73,0.014772,11.898092,10.060977,9.957698,0.025295,6.20,42.910912,10.002242,0.058531,74.8190,21.441,44.496,7.839,6.04,0.696237,0.562904,0.574395,0.574022,0.581741,0.002005,0.041476,6.132625e-02,3.185458e-02,0.09375,0.098250,0.101437,0.16750,0.161875,0.66325,0.33125,0.325750,0.042266,0.161456,15.114222,46.100803
2793,88.697849,0.23,0.016828,11.245541,9.397723,8.744774,0.019077,5.32,38.345123,8.492023,0.994723,67.7218,18.901,42.150,7.095,5.29,0.500282,0.499869,0.499749,0.500670,0.505625,0.002003,0.041449,1.670807e-05,4.734554e-05,0.00900,0.010000,0.008750,0.00550,0.007500,0.00225,0.00525,0.005250,0.000116,0.057171,16.233477,4.652433
2818,94.467810,0.26,0.045872,8.445515,8.073270,8.053166,0.049362,4.25,31.311590,7.652126,-1.313550,57.6616,12.639,33.596,6.684,3.96,0.500282,0.519559,0.539919,0.540095,0.551494,0.001993,0.041416,8.420664e-05,4.615705e-04,0.00800,0.008625,0.008688,0.01225,0.012625,0.09075,0.02050,0.023125,0.000118,0.053115,-1.006605,-1.687920
2852,99.828836,0.51,0.032406,12.322228,9.492495,9.052517,0.039012,5.74,32.863182,8.364710,2.442038,71.4062,18.643,40.147,8.069,3.83,0.854054,0.627707,0.652473,0.646980,0.656113,0.002003,0.041429,6.759214e-03,6.272943e-02,0.12475,0.131500,0.133750,0.14875,0.151375,0.70900,0.32200,0.323375,0.996464,0.744028,108.480385,84.688293
3396,47.750188,0.38,0.046994,8.490995,8.033864,7.920846,0.050957,3.80,31.698948,7.804756,-1.261830,58.0124,11.738,33.561,6.366,4.15,0.500282,0.499869,0.516241,0.515779,0.524213,0.002001,0.041672,1.656838e-03,4.288198e-03,0.00125,0.001625,0.001437,0.00475,0.004000,0.05600,0.02450,0.023750,0.000210,0.028716,-1.884505,-0.136646


## Show 54 Active Compounds' Corresponding Predicted Rank

In [5]:
rank_df = pd.read_csv('{}/complete_rank.csv'.format(dir_))

In [6]:
HTML(rank_df[rank_df['label']>0].to_html())

,molecule,label,inhibition,Baseline,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,RandomForest_h,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
367,SMSSF-0613016,1,83.123806,9,11763,7126,7020,8090,8740,9668,6313,9123,15827,1182,14795,11794,6040,5154,11259,11296,274,281,283,1968,1628,1553,1598,6,6,6,9,9,36,19,20,21499,10377,8102,3593
1021,SMSSF-0595908,1,72.063678,53,14560,18142,19423,20330,16483,10669,18428,19320,6336,12365,20011,20568,6244,22194,11259,96,63,76,64,2737,5775,2,1,69,70,70,68,60,18,29,29,9,12,38,1
1528,SMSSF-0613212,1,89.493228,1637,11114,3218,11697,13521,11860,11331,19489,2435,477,13876,14440,12389,10259,19564,64,11,13,14,14,16678,3806,561,33,1,1,1,1,1,2,1,1,3975,1899,435,186
1746,SMSSF-0596357,1,44.850520,9,1123,169,1883,2823,1557,3135,16606,15928,14530,5106,23,153,18929,694,11259,11296,11366,11511,11842,20799,19802,21770,16689,7,8,8,6,6,452,9,9,11211,12687,15760,11324
1850,SMSSF-0157653,1,93.499335,1355,2014,544,750,1288,1422,12214,7913,68,1225,5316,3462,2858,3652,7432,11259,147,97,98,95,8916,251,3138,554,142,132,140,123,117,119,62,64,134,38,329,218
2547,SMSSF-0596881,1,71.993342,1,19248,4625,12427,14563,17311,15089,17151,12849,13286,18016,2292,739,16763,13100,68,17,22,22,23,218,2764,9,6,38,33,32,13,15,8,4,4,14,22,107,10
2793,SMSSF-0613472,1,88.697849,13550,17439,11546,19299,21202,20601,19494,20632,20025,4987,20993,7959,2475,20558,17720,11259,11296,11366,11511,697,2102,4525,4899,1881,265,240,274,501,340,1472,511,510,855,92,98,986
2818,SMSSF-0613492,1,94.467810,6256,6370,22408,22113,22056,9404,21107,21851,21349,20931,22106,21734,21073,21456,21666,11259,101,78,80,68,18411,7726,1528,182,305,284,278,206,205,111,141,125,808,116,2715,6180
2852,SMSSF-0613518,1,99.828836,84,9468,2037,18688,20450,12028,17869,21686,20325,287,19824,8721,6022,14774,21838,11,2,3,3,3,3074,6259,42,4,26,24,23,18,17,5,5,5,1,1,3,2
3396,SMSSF-0613602,1,47.750188,617,6186,22401,22139,22143,9027,21328,21824,21171,20801,22079,22074,21094,21840,21404,11259,11296,165,169,167,7887,15,110,29,2385,1780,1999,597,712,187,122,122,240,1161,4067,4088


# Show n-hits in top 250

In [7]:
model_names = rank_df.columns[3:]

# Clustering Analysis

In [8]:
clustering_df = pd.read_excel('{}/clustering/lc4_clusters_2018_0802.xlsx'.format(dir_))
clustering_names = clustering_df.columns[2:].tolist()

In [9]:
merged_clustering_df = pd.merge(rank_df, clustering_df, left_on=['molecule'], right_on='Molecule')

## Clustering Number for All 54 Actives

In [10]:
for clustering_name in clustering_names:
    clustering_ids = merged_clustering_df[clustering_name].loc[positive_idx].tolist()
    clustering_num = len(set(clustering_ids))
    print clustering_name, ':\t', clustering_num

SIM_Cluster :	27
MCS_Clusters :	35


## Clustering Number for All 54 Actives

In [11]:
model_mapping_to_hits = []
baseline_active_set = set()
model_mapping_to_unique_hits = []
model_mapping_to_clustering = {}

for clustering_name in clustering_names:
    model_mapping_to_clustering[clustering_name] = []

for (idx, model_name) in enumerate(model_names):
    
    if 'ensemble' in model_name:
        order = rank_df[model_name].rank().loc[positive_idx].tolist()
        order = np.array(order)
        order = order.astype(int)
        continue
    else:
        order = rank_df[model_name].loc[positive_idx]
    
    order = rank_df[model_name].loc[positive_idx]
        
    hits_id = map(lambda x: x[0], filter(lambda x: 0<=x[1] and x[1] <=250, enumerate(order)))
    if model_name == 'Baseline':
        baseline_active_set = set(hits_id)
    current_active_set = set(hits_id)
    unique_active_set = current_active_set - baseline_active_set
    model_mapping_to_unique_hits.append(len(unique_active_set))
            
    hits = len(hits_id)
    model_mapping_to_hits.append(len(hits_id))
    
    print model_name
    for clustering_name in clustering_names:
        clustering_ids = set(merged_clustering_df[clustering_name].loc[hits_id].tolist())
        clustering_num = len(clustering_ids)
        model_mapping_to_clustering[clustering_name].append(clustering_num)
        print clustering_name, clustering_ids
    print

Baseline
SIM_Cluster set([1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14])
MCS_Clusters set([130, 38, 230, 232, 332, 16, 241, 85, 24])

ConsensusDocking_efr1_opt
SIM_Cluster set([])
MCS_Clusters set([])

ConsensusDocking_max
SIM_Cluster set([1, 3])
MCS_Clusters set([24, 230])

ConsensusDocking_mean
SIM_Cluster set([])
MCS_Clusters set([])

ConsensusDocking_median
SIM_Cluster set([])
MCS_Clusters set([])

ConsensusDocking_rocauc_opt
SIM_Cluster set([])
MCS_Clusters set([])

Docking_ad4
SIM_Cluster set([])
MCS_Clusters set([])

Docking_dock6
SIM_Cluster set([1])
MCS_Clusters set([24])

Docking_fred
SIM_Cluster set([1, 12])
MCS_Clusters set([332, 230])

Docking_hybrid
SIM_Cluster set([2])
MCS_Clusters set([38])

Docking_plants
SIM_Cluster set([1])
MCS_Clusters set([24])

Docking_rdockint
SIM_Cluster set([3])
MCS_Clusters set([230])

Docking_rdocktot
SIM_Cluster set([3])
MCS_Clusters set([230])

Docking_smina
SIM_Cluster set([])
MCS_Clusters set([])

Docking_surflex
SIM_Cluster set([1])
MCS_Cluste

In [12]:
hit_and_clustering_df = pd.DataFrame({'model_names':model_names})
hit_and_clustering_df['n-hits'] = model_mapping_to_hits
hit_and_clustering_df['Actives not in Baseline'] = model_mapping_to_unique_hits
for clustering_name in clustering_names:
    hit_and_clustering_df[clustering_name] = model_mapping_to_clustering[clustering_name]
    
hit_and_clustering_df

,model_names,n-hits,Actives not in Baseline,SIM_Cluster,MCS_Clusters
0,Baseline,32,0,12,9
1,ConsensusDocking_efr1_opt,0,0,0,0
2,ConsensusDocking_max,2,1,2,2
3,ConsensusDocking_mean,0,0,0,0
4,ConsensusDocking_median,0,0,0,0
5,ConsensusDocking_rocauc_opt,0,0,0,0
6,Docking_ad4,0,0,0,0
7,Docking_dock6,1,1,1,1
8,Docking_fred,2,1,2,2
9,Docking_hybrid,1,0,1,1


In [13]:
model_names = hit_and_clustering_df['model_names']
n_hits = hit_and_clustering_df['n-hits']
unique_n_hits = hit_and_clustering_df['Actives not in Baseline']
SIM_Cluster = hit_and_clustering_df['SIM_Cluster']
MCS_Cluster = hit_and_clustering_df['MCS_Clusters']

for a,b,c,d,e in zip(model_names, n_hits, unique_n_hits, SIM_Cluster, MCS_Cluster):
    print '{}\t&{}\t&{}\t&{}\t&{}\\\\'.format(a.replace('_', '\_'),b,c,d,e)

Baseline	&32	&0	&12	&9\\
ConsensusDocking\_efr1\_opt	&0	&0	&0	&0\\
ConsensusDocking\_max	&2	&1	&2	&2\\
ConsensusDocking\_mean	&0	&0	&0	&0\\
ConsensusDocking\_median	&0	&0	&0	&0\\
ConsensusDocking\_rocauc\_opt	&0	&0	&0	&0\\
Docking\_ad4	&0	&0	&0	&0\\
Docking\_dock6	&1	&1	&1	&1\\
Docking\_fred	&2	&1	&2	&2\\
Docking\_hybrid	&1	&0	&1	&1\\
Docking\_plants	&1	&1	&1	&1\\
Docking\_rdockint	&1	&0	&1	&1\\
Docking\_rdocktot	&1	&0	&1	&1\\
Docking\_smina	&0	&0	&0	&0\\
Docking\_surflex	&1	&1	&1	&1\\
IRV\_a	&16	&1	&10	&8\\
IRV\_b	&24	&3	&11	&8\\
IRV\_c	&28	&4	&12	&9\\
IRV\_d	&29	&4	&13	&9\\
IRV\_e	&29	&4	&13	&9\\
LSTM\_a	&1	&0	&1	&1\\
LSTM\_b	&1	&1	&1	&1\\
MultiClassification\_a	&22	&1	&12	&8\\
MultiClassification\_b	&27	&3	&13	&9\\
RandomForest\_a	&36	&6	&12	&9\\
RandomForest\_b	&37	&7	&12	&9\\
RandomForest\_c	&36	&6	&12	&9\\
RandomForest\_d	&36	&7	&12	&9\\
RandomForest\_e	&36	&7	&12	&9\\
RandomForest\_f	&24	&4	&11	&6\\
RandomForest\_g	&37	&7	&13	&9\\
RandomForest\_h	&37	&7	&13	&9\\
SingleClassifica